In [24]:
# All as One: Bernoulli NB
# Treat All Participants as One
# Try with 10% ... 90% for Training (several iterations for each)

In [25]:
use_key_features = False
shuffle_times = 100
iterations = 10
output_file = 'bernoulli_nb_results_af.csv'

In [26]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

# 1 -- Load Data

In [27]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [28]:
data = [data5, data7, data9, data11, data12, data17, data18]
all_data = pd.concat(data, ignore_index=True, sort=True)

In [29]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

all_data = all_data.drop(columns=remove)

In [30]:
# Remove rows where engagment NaN
all_data = all_data[np.isfinite(all_data['engagement'])]

# Remove rows where engagment is -1
all_data = all_data[all_data['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [31]:
# Main Columns
basic_cols = []
for i in all_data.columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [32]:
# Open Face Columns

of_cols = []
for i in all_data.columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [33]:
# Audio Columns

a_cols = []
for i in all_data.columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [34]:
# ROS Columns

ros_cols = []
for i in all_data.columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [35]:
# For Window Only:
non_window_features = []
window_features = []
for i in all_data.columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [36]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [37]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [38]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

all_data = all_data[features_to_keep]

In [39]:
# All as One Participant: Shuffle Data! 

for i in range(shuffle_times):
    all_data.reindex(np.random.permutation(all_data.index))
all_data = all_data.reset_index(drop=True)

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [40]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [41]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing 

# split_size: how much data for training
def split(split_size):
    y_data = all_data['engagement'].copy()
    X_data = all_data.drop(columns=['engagement', 'session_num', 'participant'], axis=1).copy()
        
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=1-split_size, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [42]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.naive_bayes import BernoulliNB

# Hyperparameter Combinations to Try:  

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = BernoulliNB()
    clf.fit(X_train.values, y_train.values)
    
    pred = clf.predict(X_test.values)
    scores = clf.predict_proba(X_test.values)[:,1]    
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [43]:
all_results = []
for perc in tqdm([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    print('Training Percentage:', perc)
    for n in range(iterations):
        print()
        new_results = {}
        
        # Shuffle Data For This Iteration
        for i in range(shuffle_times):
            all_data.reindex(np.random.permutation(all_data.index))        
        all_data = all_data.reset_index(drop=True)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(perc)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)

        # Evaluate on Test Using Best Hyperaparemeters  
        best = {}
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_percentage'] = perc
        results['iteration'] = n
        results['hyperparameters'] = json.dumps(best)
        all_results.append(results)

  0%|          | 0/11 [00:00<?, ?it/s]

Training Percentage: 0.01

Best HP {}
{'recall_0': 0.6920241131463019, 'precision_0': 0.6419261871988988, 'precision_1': 0.8296024527599964, 'f1_1': 0.8120699181274802, 'auc': 0.7942104729412234, 'recall_1': 0.7952630996446095, 'f1_0': 0.6660344096579117, 'accuracy': 0.7594838046107241}

Best HP {}
{'recall_0': 0.6577354026028255, 'precision_0': 0.6432606574857636, 'precision_1': 0.8164286069775544, 'f1_1': 0.8115314691368711, 'auc': 0.7630171459655206, 'recall_1': 0.8066927294632539, 'f1_0': 0.6504175077995963, 'accuracy': 0.755096465218684}

Best HP {}
{'recall_0': 0.6749188687992582, 'precision_0': 0.6393921426532271, 'precision_1': 0.8222830494728305, 'f1_1': 0.8099812134489237, 'auc': 0.7682692283624173, 'recall_1': 0.7980420371668573, 'f1_0': 0.6566753498652413, 'accuracy': 0.755361634083038}

Best HP {}
{'recall_0': 0.674823714974949, 'precision_0': 0.6378784888947356, 'precision_1': 0.8221841704718417, 'f1_1': 0.8093671376040255, 'auc': 0.7682109959650427, 'recall_1': 0.7969435

  9%|▉         | 1/11 [00:56<09:22, 56.28s/it]

{'recall_0': 0.6654683767423522, 'precision_0': 0.6434417957975467, 'precision_1': 0.8193748747745943, 'f1_1': 0.8118718515025931, 'auc': 0.7644240541640772, 'recall_1': 0.8045049918851128, 'f1_0': 0.654269752593775, 'accuracy': 0.756333919919003}
Training Percentage: 0.05

Best HP {}
{'recall_0': 0.6708989334687658, 'precision_0': 0.6427479147358666, 'precision_1': 0.8215601683691534, 'f1_1': 0.8119196801720955, 'auc': 0.772815440305763, 'recall_1': 0.8025028179116713, 'f1_0': 0.6565217905785182, 'accuracy': 0.7569355473492937}

Best HP {}
{'recall_0': 0.6788915464714551, 'precision_0': 0.640660267663756, 'precision_1': 0.8241000529380624, 'f1_1': 0.8108470603554919, 'auc': 0.7748437285068788, 'recall_1': 0.7980135845187748, 'f1_0': 0.6592220710364567, 'accuracy': 0.7567262039339815}

Best HP {}
{'recall_0': 0.6803874676908955, 'precision_0': 0.6431621492019273, 'precision_1': 0.8250515682022531, 'f1_1': 0.8121859787051259, 'auc': 0.7803597866010166, 'recall_1': 0.7997154721624116, 'f

 18%|█▊        | 2/11 [01:49<08:19, 55.51s/it]

...

{'recall_0': 0.6784062885326757, 'precision_0': 0.6434000877064756, 'precision_1': 0.8263420724094881, 'f1_1': 0.8143788705245458, 'auc': 0.7748639645133497, 'recall_1': 0.8027571151358344, 'f1_0': 0.6604396428839372, 'accuracy': 0.7599703012303776}
Training Percentage: 0.8

Best HP {}
{'recall_0': 0.6780074314909429, 'precision_0': 0.6391899288451013, 'precision_1': 0.8267632910601612, 'f1_1': 0.8134738912622553, 'auc': 0.7740053904159955, 'recall_1': 0.8006049606775559, 'f1_0': 0.6580267087395053, 'accuracy': 0.7586110890143982}

Best HP {}
{'recall_0': 0.6672072280783041, 'precision_0': 0.6386517352256348, 'precision_1': 0.8218073559511258, 'f1_1': 0.8120862956607011, 'auc': 0.7698463664575701, 'recall_1': 0.8025925252892362, 'f1_0': 0.6526172671651936, 'accuracy': 0.7561053217723331}

Best HP {}
{'recall_0': 0.6775500752576126, 'precision_0': 0.6409638554216868, 'precision_1': 0.8260679490382213, 'f1_1': 0.8135436848417751, 'auc': 0.7750473584741076, 'recall_1': 0.8013935171160255,

 91%|█████████ | 10/11 [08:54<00:53, 53.02s/it]

{'recall_0': 0.6680914972273567, 'precision_0': 0.6375964718853363, 'precision_1': 0.8212419113987058, 'f1_1': 0.810799189139382, 'auc': 0.7689917976110576, 'recall_1': 0.8006187067815116, 'f1_0': 0.6524878709240663, 'accuracy': 0.7549916474425264}
Training Percentage: 0.9

Best HP {}
{'recall_0': 0.6769983686786297, 'precision_0': 0.6394453004622496, 'precision_1': 0.827354260089686, 'f1_1': 0.814569536423841, 'auc': 0.7750259502812875, 'recall_1': 0.8021739130434783, 'f1_0': 0.6576862123613313, 'accuracy': 0.7594463447617532}

Best HP {}
{'recall_0': 0.6739811912225705, 'precision_0': 0.6615384615384615, 'precision_1': 0.8184764991896273, 'f1_1': 0.8142130720575468, 'auc': 0.7752647393024767, 'recall_1': 0.8099938309685379, 'f1_0': 0.6677018633540373, 'accuracy': 0.7616736934213666}

Best HP {}
{'recall_0': 0.6767630057803469, 'precision_0': 0.6385253054101222, 'precision_1': 0.8249655690497057, 'f1_1': 0.8118031171071275, 'auc': 0.7688527771040504, 'recall_1': 0.7990540868299781, 'f

100%|██████████| 11/11 [09:47<00:00, 53.06s/it]

{'recall_0': 0.6611834589901759, 'precision_0': 0.6338151554971528, 'precision_1': 0.8147407870081199, 'f1_1': 0.8052348910426569, 'auc': 0.7632853106545971, 'recall_1': 0.7959482548206004, 'f1_0': 0.647210108464721, 'accuracy': 0.7490255349614191}


In [44]:
results = pd.DataFrame(columns=['train_percentage', 'iteration', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'hyperparameters'])
results = results.append(all_results, ignore_index=True, sort=True)

In [45]:
results.head()

,accuracy,auc,f1_0,f1_1,hyperparameters,iteration,precision_0,precision_1,recall_0,recall_1,train_percentage
0,0.759484,0.794210,0.666034,0.812070,{},0,0.641926,0.829602,0.692024,0.795263,0.01
1,0.755096,0.763017,0.650418,0.811531,{},1,0.643261,0.816429,0.657735,0.806693,0.01
2,0.755362,0.768269,0.656675,0.809981,{},2,0.639392,0.822283,0.674919,0.798042,0.01
3,0.754638,0.768211,0.655831,0.809367,{},3,0.637878,0.822184,0.674824,0.796944,0.01
4,0.759685,0.791624,0.669028,0.811356,{},4,0.640104,0.832805,0.700688,0.790985,0.01


In [46]:
results.to_csv(output_file, index=False)